In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 62.2 MB/s eta 0:00:00


In [ ]:
## reference https://github.com/huggingface/trl/blob/main/examples/research_projects/toxicity/scripts/gpt-j-6b-toxicity.py

In [61]:
import torch
from torch.optim import Adam
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from datasets import load_dataset
import transformers

In [60]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, create_reference_model, set_seed

from trl.core import LengthSampler

In [11]:
model_name = "allenai/OLMo-1B"

In [12]:
config = PPOConfig(
    model_name = model_name,
    learning_rate = (1.47e-5) * 2,
    ppo_epochs=100,
    mini_batch_size = 8,
    batch_size = 32,
    gradient_accumulation_steps=4
)

In [14]:
config.ppo_epochs

100

In [26]:
def build_dataset(config, dataset_name="allenai/real-toxicity-prompts", input_min_text_length=5, input_max_text_length=10
):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token

    ds = load_dataset(dataset_name, split="train")

    def filter_fn(sample):
        toxicity = sample["prompt"]["toxicity"]
        return toxicity is not None and toxicity > 0.3

    ds = ds.filter(filter_fn, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        prompt = sample["prompt"]["text"]
        continuation = sample["continuation"]["text"]

        sample["input_ids"] = tokenizer.encode(prompt + continuation)[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")

    ds = ds.train_test_split(test_size=0.2, shuffle=False)["train"]

    return ds

In [27]:
min_input_length = 30
max_input_length = 40
dataset = build_dataset(config,"allenai/real-toxicity-prompts", input_min_text_length=min_input_length, input_max_text_length=max_input_length)

Filter:   0%|          | 0/99442 [00:00<?, ? examples/s]

Map:   0%|          | 0/35109 [00:00<?, ? examples/s]

In [28]:
def collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

In [68]:

model = AutoModelForCausalLM.from_pretrained("ybelkada/gpt-j-6b-sharded-bf16", torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [69]:
model

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(model)

In [40]:
type(model)

hf_olmo.modeling_olmo.OLMoForCausalLM

In [63]:
class OlmoAutoModelForCausalLMWithValueHead(AutoModelForCausalLMWithValueHead):
    lm_head_namings = ["ff_out"]



In [67]:
OlmoAutoModelForCausalLMWithValueHead.lm_head_namings

['ff_out']

In [66]:
set_seed(config.seed)

# Now let's build the model, the reference model, and the tokenizer. We first load the model
# in bfloat16 to save memory using `transformers`.
model = AutoModelForCausalLM.from_pretrained(config.model_name, torch_dtype=torch.bfloat16)
# And then we pass the loaded model to `AutoModelForCausalLMWithValueHead`.
model = OlmoAutoModelForCausalLMWithValueHead.from_pretrained(model)

ValueError: The model does not have a language model head, please use a model that has one.